# text-embeddings-ada-002 - via OpenAI Python library

SPDX-License-Identifier: 0BSD

In [1]:
import functools
import itertools

from colorama import Back, Fore, Style
import numpy as np
import openai
import tiktoken

In [2]:
openai.api_key_path = '.openai_key'

In [3]:
texts = [
    'That is a happy person',
    'That is a happy dog',
    'That is a very happy person',
    'Today is a sunny day',
]

In [4]:
results = openai.Embedding.create(
    input=texts,
    model='text-embedding-ada-002',
)

In [5]:
results.data[0].keys()

dict_keys(['object', 'index', 'embedding'])

In [6]:
[datum.object for datum in results.data]

['embedding', 'embedding', 'embedding', 'embedding']

In [7]:
[datum.index for datum in results.data]

[0, 1, 2, 3]

In [8]:
pairs = itertools.pairwise(datum.index for datum in results.data)
all(lhs < rhs for lhs, rhs in pairs)

True

In [9]:
v1, v2, v3, v4 = (
    np.array(datum.embedding, dtype=np.float32)
    for datum in results.data
)

In [10]:
[embedding.shape for embedding in (v1, v2, v3, v4)]

[(1536,), (1536,), (1536,), (1536,)]

In [11]:
[np.linalg.norm(embedding) for embedding in (v1, v2, v3, v4)]

[1.0, 0.99999994, 1.0000001, 1.0]

In [12]:
np.dot(v1, v2)

0.93006015

In [13]:
np.dot(v1, v3)

0.9834069

In [14]:
np.dot(v1, v4)

0.822598

In [15]:
with open('the_open_window.txt', encoding='utf-8') as file:
    tow = file.read().strip().replace('\n', ' ')

with open('the_open_window_modified.txt', encoding='utf-8') as file:
    tow_modified = file.read().strip().replace('\n', ' ')

assert tow != tow_modified

tow_results = openai.Embedding.create(
    input=[tow, tow_modified],
    model='text-embedding-ada-002',
)

In [16]:
[datum.object for datum in tow_results.data]

['embedding', 'embedding']

In [17]:
[datum.index for datum in tow_results.data]

[0, 1]

In [18]:
pairs = itertools.pairwise(datum.index for datum in tow_results.data)
all(lhs < rhs for lhs, rhs in pairs)

True

In [19]:
w1, w2 = (
    np.array(datum.embedding, dtype=np.float32)
    for datum in tow_results.data
)

In [20]:
(w1 == w2).all()

False

In [21]:
np.linalg.norm(w1 - w2)

0.08120534

In [22]:
np.dot(w1, w2)

0.99670285

In [23]:
results.usage

<OpenAIObject at 0x7f4a9b7b6b70> JSON: {
  "prompt_tokens": 21,
  "total_tokens": 21
}

In [24]:
tow_results.usage

<OpenAIObject at 0x7f4a9b7b4ef0> JSON: {
  "prompt_tokens": 3114,
  "total_tokens": 3114
}

In [25]:
enc = tiktoken.get_encoding('cl100k_base')

In [26]:
enc.encode('That is a very happy person')

[4897, 374, 264, 1633, 6380, 1732]

In [27]:
enc.encode('altogether doctrinaire')

[76777, 3522, 61990, 68976]

In [28]:
_COLORS = (
    Style.BRIGHT + Fore.BLACK + Back.LIGHTGREEN_EX,
    Style.BRIGHT + Fore.BLACK + Back.LIGHTMAGENTA_EX,
)

In [29]:
def _index_color(index):
    return _COLORS[index % len(_COLORS)]

In [30]:
def _do_reveal(enc, text):
    pieces = (enc.decode([code], errors='strict') for code in enc.encode(text))
    pretty = (_index_color(index) + piece for index, piece in enumerate(pieces))
    print(*pretty, sep='', end=(Style.RESET_ALL + '\n'))

In [31]:
def make_token_revealer(encoding_name):
    enc = tiktoken.get_encoding(encoding_name)
    return functools.partial(_do_reveal, enc)

In [32]:
reveal_cl100k_base = make_token_revealer('cl100k_base')
reveal_gpt2 = make_token_revealer('gpt2')

In [33]:
def reveal_both(text):
    reveal_cl100k_base(text)
    reveal_gpt2(text)

In [34]:
reveal_both('altogether doctrinaire')

altogether doctrinaire
altogether doctrinaire


In [35]:
reveal_both('Dr. Von Squilldebrandt')

Dr. Von Squilldebrandt
Dr. Von Squilldebrandt


In [36]:
reveal_both('eberschrifvt')

eberschrifvt
eberschrifvt


In [37]:
reveal_both('wavelet bah')

wavelet bah
wavelet bah


In [38]:
reveal_both('Fourscore and seven years ago')

Fourscore and seven years ago
Fourscore and seven years ago


In [39]:
reveal_both('Our fiendish foes followed in force, onagers a-ready.')

Our fiendish foes followed in force, onagers a-ready.
Our fiendish foes followed in force, onagers a-ready.


In [40]:
reveal_both('"Squee!" said one mouse. "Squick!" said the other.')

"Squee!" said one mouse. "Squick!" said the other.
"Squee!" said one mouse. "Squick!" said the other.


In [41]:
reveal_both('The butler subtly buttled with a glut of scuttlebutt cutlery.')

The butler subtly buttled with a glut of scuttlebutt cutlery.
The butler subtly buttled with a glut of scuttlebutt cutlery.


In [42]:
reveal_both('Patients were assessed electroencephalographically.')

Patients were assessed electroencephalographically.
Patients were assessed electroencephalographically.


In [43]:
reveal_both('"Isobaric" means "having equal pressure."')

"Isobaric" means "having equal pressure."
"Isobaric" means "having equal pressure."


In [44]:
reveal_both("I'm afraid we've sinned rather unforgivably, old chap.")

I'm afraid we've sinned rather unforgivably, old chap.
I'm afraid we've sinned rather unforgivably, old chap.


In [45]:
reveal_both('In the Shakespearean tragedy *Othello*, Brabantio was the father of Desdemona.')

In the Shakespearean tragedy *Othello*, Brabantio was the father of Desdemona.
In the Shakespearean tragedy *Othello*, Brabantio was the father of Desdemona.


In [46]:
reveal_both('The Slopes Bureau frowns on whalebone skis, for reasons of conservation.')

The Slopes Bureau frowns on whalebone skis, for reasons of conservation.
The Slopes Bureau frowns on whalebone skis, for reasons of conservation.


In [47]:
reveal_both("The ski instructor upbraided the novice for attempting a black diamond, which was beyond the student's capabilities and also triggered an unfortunate avalanche.")

The ski instructor upbraided the novice for attempting a black diamond, which was beyond the student's capabilities and also triggered an unfortunate avalanche.
The ski instructor upbraided the novice for attempting a black diamond, which was beyond the student's capabilities and also triggered an unfortunate avalanche.


In [48]:
reveal_both('Alpha Centauri is the nearest star to our sun.')

Alpha Centauri is the nearest star to our sun.
Alpha Centauri is the nearest star to our sun.


In [49]:
reveal_both('Lana left the sonic harmonica on the Encyclopaedia Daemonica.')

Lana left the sonic harmonica on the Encyclopaedia Daemonica.
Lana left the sonic harmonica on the Encyclopaedia Daemonica.


In [50]:
reveal_both('dtunqhoepgfxvzcwlsajkmbiry')

dtunqhoepgfxvzcwlsajkmbiry
dtunqhoepgfxvzcwlsajkmbiry


In [51]:
def count_tokens(text):
    return len(enc.encode(text))

In [52]:
texts

['That is a happy person',
 'That is a happy dog',
 'That is a very happy person',
 'Today is a sunny day']

In [53]:
sum(count_tokens(text) for text in texts)

21

In [54]:
count_tokens(tow) + count_tokens(tow_modified)

3114

In [55]:
reveal_cl100k_base(tow)

"My aunt will be down presently, Mr. Nuttel," said a very self-possessed young lady of fifteen; "in the meantime you must try and put up with me."  Framton Nuttel endeavored to say the correct something which should duly Hatter the niece of the moment without unduly discounting the aunt that was to come. Privately he doubted more than ever whether these formal visits on a succession of total strangers would do very much towards helping the nerve cure which he was supposed to be undergoing.  "I know how it will be," his sister had said when he was preparing to migrate to this rural retreat; "you will bury yourself down there and not speak to a living soul, and your nerves will be worse than ever from moping.  I shall just give you letters of introduction to all the people I know there. Some of them, as far as I can remember, were quite nice."  Framton wondered whether Mrs. Sappleton, the lady to whom he was presenting one of the letters of introduction came into the nice division.  "Do 

In [56]:
reveal_gpt2(tow)

"My aunt will be down presently, Mr. Nuttel," said a very self-possessed young lady of fifteen; "in the meantime you must try and put up with me."  Framton Nuttel endeavored to say the correct something which should duly Hatter the niece of the moment without unduly discounting the aunt that was to come. Privately he doubted more than ever whether these formal visits on a succession of total strangers would do very much towards helping the nerve cure which he was supposed to be undergoing.  "I know how it will be," his sister had said when he was preparing to migrate to this rural retreat; "you will bury yourself down there and not speak to a living soul, and your nerves will be worse than ever from moping.  I shall just give you letters of introduction to all the people I know there. Some of them, as far as I can remember, were quite nice."  Framton wondered whether Mrs. Sappleton, the lady to whom he was presenting one of the letters of introduction came into the nice division.  "Do 

In [57]:
with open('the_open_window.txt', encoding='utf-8') as file:
    tow_nl = file.read()

In [58]:
paragraphs = (
    raw_graf.replace('\n', ' ')
    for raw_graf in tow_nl.split('\n\n')
)

for graf in paragraphs:
    print()
    reveal_cl100k_base(graf)
    print()
    reveal_gpt2(graf)
    print()


"My aunt will be down presently, Mr. Nuttel," said a very self-possessed young lady of fifteen; "in the meantime you must try and put up with me."

"My aunt will be down presently, Mr. Nuttel," said a very self-possessed young lady of fifteen; "in the meantime you must try and put up with me."


Framton Nuttel endeavored to say the correct something which should duly Hatter the niece of the moment without unduly discounting the aunt that was to come. Privately he doubted more than ever whether these formal visits on a succession of total strangers would do very much towards helping the nerve cure which he was supposed to be undergoing.

Framton Nuttel endeavored to say the correct something which should duly Hatter the niece of the moment without unduly discounting the aunt that was to come. Privately he doubted more than ever whether these formal visits on a succession of total strangers would do very much towards helping the nerve cure which he was supposed to be undergoing.


"I kn